# Circos Genomic Visualization Workflow - Master Notebook

This notebook orchestrates the complete Circos visualization pipeline for genomic data analysis. The workflow consists of 5 sequential steps plus specialized miRNA analysis.

## Workflow Overview

1. **Step 1: Stacked Histograms** - Basic multi-dimensional genomic data visualization
2. **Step 2: Label Practice** - Annotation and labeling techniques
3. **Step 3: Complex Histograms** - Advanced multi-track visualizations
4. **Step 4: PCA & LMD Contextual** - Principal Component Analysis with genomic context
5. **Step 5: Trisomy Contextual** - Chromosomal aberration analysis
6. **miRNA Analysis** - Specialized microRNA target analysis

## Prerequisites

- Python 3.8+
- Circos software installed
- Required Python packages (see requirements below)

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting parameters
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Master Workflow Environment Setup Complete")
print(f"Working directory: {os.getcwd()}")

Matplotlib is building the font cache; this may take a moment.


Master Workflow Environment Setup Complete
Working directory: /Users/mbp-14/Downloads/CLONED/Circos_Genomic_Data_Visualization


In [2]:
# Configuration and global variables
BASE_DIR = Path(os.getcwd())
WORKFLOWS = {
    1: "1_stackedHistograms",
    2: "2_labelPractice", 
    3: "3_complexHistograms",
    4: "4_PCA_LMD_contextual",
    5: "5_TrisomyContextual",
    "miRNA": "miRNA"
}

# Check if Circos is installed
def check_circos_installation():
    try:
        result = subprocess.run(['circos', '-version'], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✓ Circos installed: {result.stdout.strip()}")
            return True
        else:
            print("✗ Circos not found in PATH")
            return False
    except FileNotFoundError:
        print("✗ Circos not installed or not in PATH")
        return False

circos_available = check_circos_installation()

# Display workflow directories
print("\nAvailable Workflows:")
for key, dirname in WORKFLOWS.items():
    path = BASE_DIR / dirname
    status = "✓" if path.exists() else "✗"
    print(f"{status} Step {key}: {dirname}")

✗ Circos not installed or not in PATH

Available Workflows:
✓ Step 1: 1_stackedHistograms
✓ Step 2: 2_labelPractice
✓ Step 3: 3_complexHistograms
✓ Step 4: 4_PCA_LMD_contextual
✓ Step 5: 5_TrisomyContextual
✓ Step miRNA: miRNA


## Workflow Execution

Run the cells below to execute individual workflow steps. Each step can be run independently or as part of the complete pipeline.

In [ ]:
def run_workflow_step(step_number, execute_circos=True):
    """
    Execute a specific workflow step
    
    Parameters:
    step_number: int or str - The workflow step to execute
    execute_circos: bool - Whether to run Circos after data preparation
    """
    if step_number not in WORKFLOWS:
        print(f"Error: Step {step_number} not found")
        return False
        
    workflow_dir = BASE_DIR / WORKFLOWS[step_number]
    if not workflow_dir.exists():
        print(f"Error: Directory {workflow_dir} does not exist")
        return False
        
    print(f"\n{'='*50}")
    print(f"Executing Step {step_number}: {WORKFLOWS[step_number]}")
    print(f"{'='*50}")
    
    # Change to workflow directory
    original_dir = os.getcwd()
    os.chdir(workflow_dir)
    
    try:
        # Execute the corresponding notebook
        notebook_name = f"Step_{step_number}_Workflow.ipynb"
        if (workflow_dir / notebook_name).exists():
            print(f"Running notebook: {notebook_name}")
            # Here you would typically use nbconvert or papermill to execute
            # For now, we'll just indicate the notebook should be run
            print(f"→ Please run {notebook_name} in {workflow_dir}")
        else:
            print(f"Notebook {notebook_name} not found. Creating...")
            
        # If Circos execution is requested and available
        if execute_circos and circos_available:
            circos_conf = workflow_dir / "circos.conf"
            if circos_conf.exists():
                print("Executing Circos...")
                result = subprocess.run(['circos', '-conf', 'circos.conf'], 
                                      capture_output=True, text=True)
                if result.returncode == 0:
                    print("✓ Circos execution successful")
                    # Check for output files
                    for ext in ['png', 'svg']:
                        output_file = workflow_dir / f"circos.{ext}"
                        if output_file.exists():
                            print(f"  → Generated: {output_file}")
                else:
                    print(f"✗ Circos execution failed: {result.stderr}")
            else:
                print(f"No circos.conf found in {workflow_dir}")
                
    except Exception as e:
        print(f"Error executing step {step_number}: {e}")
        return False
    finally:
        # Return to original directory
        os.chdir(original_dir)
        
    print(f"Step {step_number} completed\n")
    return True

# Test function
print("Workflow execution function ready")

In [ ]:
# Execute individual steps (uncomment to run)

# Step 1: Stacked Histograms
# run_workflow_step(1)

# Step 2: Label Practice  
# run_workflow_step(2)

# Step 3: Complex Histograms
# run_workflow_step(3)

# Step 4: PCA & LMD Contextual
# run_workflow_step(4)

# Step 5: Trisomy Contextual
# run_workflow_step(5)

# miRNA Analysis
# run_workflow_step("miRNA")

print("Individual workflow steps ready to execute")
print("Uncomment the desired step above to run")

In [ ]:
# Execute complete workflow pipeline
def run_complete_pipeline():
    """
    Execute all workflow steps in sequence
    """
    print("Starting Complete Circos Workflow Pipeline")
    print("="*60)
    
    success_count = 0
    total_steps = len(WORKFLOWS)
    
    for step in [1, 2, 3, 4, 5, "miRNA"]:
        if run_workflow_step(step, execute_circos=True):
            success_count += 1
        else:
            print(f"Pipeline halted at step {step}")
            break
            
    print(f"\nPipeline Summary: {success_count}/{total_steps} steps completed successfully")
    
    if success_count == total_steps:
        print("🎉 Complete pipeline executed successfully!")
    else:
        print("⚠️  Pipeline completed with some failures")

# Uncomment to run complete pipeline
# run_complete_pipeline()

print("Complete pipeline function ready")
print("Uncomment run_complete_pipeline() above to execute all steps")

## Utility Functions

Helper functions for data processing and visualization

In [ ]:
# Utility functions for data processing

def load_genomic_data(file_path, file_type='tab'):
    """
    Load genomic data from various file formats
    """
    try:
        if file_type == 'tab':
            return pd.read_csv(file_path, sep='\t', header=None)
        elif file_type == 'csv':
            return pd.read_csv(file_path)
        else:
            return pd.read_csv(file_path, sep='\t')
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def create_circos_histogram_data(data, chromosomes, bin_size=5000000):
    """
    Create histogram data in Circos format
    """
    histogram_data = []
    
    for chrom in chromosomes:
        chrom_data = data[data['chromosome'] == chrom] if 'chromosome' in data.columns else data
        
        # Create bins and calculate values
        max_pos = chrom_data['position'].max() if 'position' in chrom_data.columns else 250000000
        
        for start in range(0, int(max_pos), bin_size):
            end = min(start + bin_size - 1, int(max_pos))
            
            # Calculate histogram values for this bin
            bin_data = chrom_data[
                (chrom_data['position'] >= start) & 
                (chrom_data['position'] <= end)
            ] if 'position' in chrom_data.columns else chrom_data
            
            # Generate sample values (replace with actual calculation)
            values = [np.random.random() for _ in range(5)]
            value_str = ','.join([f"{v:.6f}" for v in values])
            
            histogram_data.append(f"{chrom} {start} {end} {value_str}")
    
    return histogram_data

def generate_circos_config(workflow_name, data_files):
    """
    Generate Circos configuration file
    """
    config = f"""
<<include etc/colors_fonts_patterns.conf>>
<<include ideogram.conf>>
<<include ticks.conf>>

karyotype = data/karyotype/karyotype.human.txt

<image>
<<include etc/image.conf>>
</image>

chromosomes_units = 1000000
chromosomes = /hs/
chromosomes_display_default = no

<plots>
"""
    
    # Add plot configurations based on data files
    for i, data_file in enumerate(data_files):
        config += f"""
<plot>
file = {data_file}
type = histogram
r0 = {0.6 + i*0.1}r
r1 = {0.8 + i*0.1}r
min = 0
max = 1
color = white
fill_color = red,orange,yellow,green,blue
thickness = 2
</plot>
"""
    
    config += """
</plots>

<<include etc/housekeeping.conf>>
"""
    
    return config

print("Utility functions loaded successfully")

## Summary and Next Steps

This master workflow notebook provides:

1. **Centralized Control** - Execute all workflow steps from one location
2. **Modern Python Environment** - Replace Perl scripts with Python/Pandas
3. **Jupyter Integration** - Interactive analysis and visualization
4. **Modular Design** - Each step can be run independently
5. **Error Handling** - Robust execution with proper error reporting

### To Complete the Migration:

1. Create individual workflow notebooks for each step (1-5 + miRNA)
2. Implement data processing functions specific to each analysis type
3. Add visualization capabilities using matplotlib/seaborn
4. Create requirements.txt for Python dependencies
5. Test and validate against original Perl output

### Next: Run Individual Workflow Creation

Execute the cells above to create individual notebooks for each workflow step.